In [1]:
import ee 

ee.Initialize()

In [3]:
import geemap
from sepal_ui.scripts import mapping as sepalmap

Map = sepalmap.init_map()[1]
Map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [3]:
#create a false list of points
import shapely.geometry as sg 
from faker import Faker
from decimal import Decimal
import geopandas

Faker.seed(0)
fake = Faker()

points_nb = 4

pts = []
for i in range(points_nb):
    point = sg.Point(fake.longitude(), fake.latitude())
    pts.append(point)

ee_pts = []
for index, pt in enumerate(pts):
    #transform shape in geojson
    json = geopandas.GeoSeries(pt).__geo_interface__
    #transform gson in ee_feature 
    ee_object = geemap.geojson_to_ee(json)
    #add to the array 
    ee_pts.append(ee_object)
    #display on the map
    Map.addLayer(ee_object, {}, str(index))
    
ee_pts[1].geometry().getInfo()

{'type': 'Point', 'coordinates': [-158.265114, -20.4975645]}

In [4]:
#define the cloud masking function 


def clouMask_landsat(image):
    """
    Function to mask clouds based on the pixel_qa band of Landsat SR data.
    
    Args:
        image(ee.Image): image Input Landsat SR image
    Returns:
        (ee.Image): Cloudmasked Landsat image
    """
    qa = image.select('pixel_qa')
    
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    
    return image.updateMask(cloud.Not()).updateMask(mask2)

def cloudMask_sentinel(image):
    """
    Function to mask clouds using the Sentinel-2 QA band
    
    Args:
        {ee.Image} image Sentinel-2 image
    Returns:
        {ee.Image} cloud masked Sentinel-2 image
    """
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    
    #  Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask).divide(10000)


    

In [5]:
#define the visualiztion parameters for both the sentinel and the landsat data

visParams_landsat = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}


visParams_sentinel = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

In [6]:
annual_time_series = []
#for each point create a dictionnary with all the cliped images required for the final display
index = 0
point = pts[index]
annual_time_series.append({})

#create un buffer
size = 10000

buffer = ee_pts[index].geometry().buffer(size)

Map.addLayer(buffer, {'color': 'green'}, 'buffer')
Map.centerObject(buffer, zoom=10)

#load the landsat data
landsat_start = 2005
landsat_end = 2015

for year in range(landsat_start, landsat_end + 1):
    start = '{}-01-01'.format(year)
    end = '{}-12-31'.format(year)
    
    dataset = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate(start,end)#.map(clouMask_landsat)
        
    clip = dataset.median().clip(buffer)
    
    annual_time_series[0][year] = clip
    
Map.addLayer(annual_time_series[0][2008], visParams_landsat, 'landsat-{}'.format(2008))
    
#load the sentinel data 
sentinel_start = 2016
sentinel_end = 2019

for year in range(sentinel_start, sentinel_end + 1):
    start = '{}-01-01'.format(year)
    end = '{}-12-31'.format(year)
    
    dataset = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(start, end).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))#.map(cloudMask_sentinel)

    clip = dataset.mean().clip(buffer)

    annual_time_series[0][year] = clip
    
    #export the image : 
    #task_config = {
    #    'image':clip,
    #    'description':'test-{}'.format(year),
    #    'scale': 30,
    #    'region':buffer,
    #    'maxPixels': 1e12
    #}
    
    #task = ee.batch.Export.image.toDrive(**task_config)
    #task.start()
    
Map.addLayer(annual_time_series[0][2017], visParams_sentinel, 'sentinel-{}'.format(2017))